### 1. Explain the difference between “__new__” and “__init__” in Python class instantiation, and provide a scenario where overriding new is necessary (e.g., for implementing a singleton pattern).

- `__new__` is a **static method** responsible for **creating a new instance** of a class. → useful for singleton for example.
- It is called **before** `__init__`.
- `__init__` is an **instance method** that initializes the newly created object.

In [5]:
# singleton

class Singleton:
	_instance = None
	def __new__(cls, *args, **kwargs):
		if cls._instance is None:
			cls._instance = super().__new__(cls)
		return cls._instance
		
	def __init__(self, value, *args, **kwargs):
		if not hasattr(self, "_initialized"):
			self.value = value
			self._initialized = True
			
			
# usage
s1 = Singleton(10)
s2 = Singleton(20)

print(s1.value)    
print(s2.value)    

10
10


---

### 2. What are metaclasses in Python? Write a custom metaclass that automatically registers all subclasses and enforces that every class has a specific method (e.g., validate()). Discuss real-world uses, such as in ORM frameworks like Django.

metaclass → classes of classes

- **a metaclass controls class creation and modification**.

1. **Instance** → an object of a class.
2. **Class** → defines how instances behave.
3. **Metaclass** → defines how classes behave.
- In Python, **everything is an object**.
- Classes themselves are **instances of a metaclass**.

• The default metaclass is `type`:

```python
class MyClass:
	pass
	
print(type(MyClass))   # <class 'type'>
```


**Key Idea:** Metaclasses let you **intercept class creation** and customize it.

A metaclass usually overrides **`__new__`** or **`__init__`**

In [ ]:
class MyMeta(type):
	def __new__(cls, name, bases, namespace):
		print(f"Creating class {name}")
		return super().__new__(cls, name, bases, namespace)
		
class Person(metaclass=MyMeta):
	def greet(self):
		print("Hello")
		
# Output: Creating class Person -> when the class is defined (this is shown when we just define the class!)

Creating class Person


`MyMeta.__new__` is called **when the class is defined**, not when instances are created.

In [ ]:
# another example of meta class
class RegistryMeta(type):
    registry = []

    def __new__(cls, name, bases, namespace):
        # Enforce 'validate' method
        if 'validate' not in namespace:
            raise TypeError(f"Class {name} must implement a 'validate()' method")
        
        # Create the class
        new_class = super().__new__(cls, name, bases, namespace)
        
        # Register the class
        cls.registry.append(new_class)
        return new_class

# Base class
class Base(metaclass=RegistryMeta):
    def validate(self):
        pass

# Subclasses
class User(Base):
    def validate(self):
        print("Validating user")

class Product(Base):
    def validate(self):
        print("Validating product")

print(RegistryMeta.registry)

[<class '__main__.Base'>, <class '__main__.User'>, <class '__main__.Product'>]


Singleton Pattern with metaclass:

In [7]:
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Logger(metaclass=SingletonMeta):
    pass

a = Logger()
b = Logger()
print(a is b)  # True

True


at first it is confusing. but just consider the order of execution.
Ultra-Short Rule (Memorize This)

class → metaclass `__new__()` → metaclass `__call__` → class `__new__` → class `__init__`

#### Real-World Uses of Metaclasses

1. ORM Frameworks (Django, SQLAlchemy)

- Django’s ModelBase metaclass scans model fields and prepares database mappings.
- You don’t have to write SQL manually; metaclass handles it.

2. Plugin Systems

- Automatically register subclasses, so plugins are discoverable without extra boilerplate.

3. Singleton Pattern (as i wrote above)

4. Enforcing Class Interfaces

Automatically ensure certain methods or attributes(like ORM fields) exist on all subclasses (like our validate() example).